Importing Necessary Libraries for Image Processing and creating a CNN Model

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import _keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

Address of Training and Validation Directory

In [ ]:
train_dir = '/content/drive/MyDrive/Colab Notebooks/FakeFaceDetection/FakeFaces/Train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/FakeFaceDetection/FakeFaces/Validation'

Image Processing

In [ ]:
train_datagen = ImageDataGenerator( rescale = 1.0/ 255.0, rotation_range = 0,
                                    width_shift_range = 0.05, height_shift_range = 0.05,
                                    shear_range = 0.0, zoom_range = 0.1,
                                    horizontal_flip = True, fill_mode = 'nearest')

val_datagen = ImageDataGenerator( rescale = 1.0/255.0)

In [ ]:
train_generator = train_datagen.flow_from_directory( train_dir, target_size = (300, 300), batch_size = 64, class_mode = 'categorical', shuffle = True)
validation_generator = val_datagen.flow_from_directory( val_dir, target_size = (300, 300), batch_size = 64, class_mode = 'categorical', shuffle = False)

Convolutional Neural Network (CNN) Structure 


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(300, 300, 3)),  #first layer of the CNN
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),    #second layer of the CNN
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),    #third layer of the CNN
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),     #final layer of CNN which is fully connected 
    tf.keras.layers.Dense(2, activation='softmax')     #output layer of CNN with 2 output classes, 'real' or 'fake'
])

model.summary()

Compilation of Model mentioning the loss, optimizer and metrics

In [ ]:
model.compile(loss= 'categorical_crossentropy', optimizer= 'adam', metrics= ['accuracy'])

Training the Model on Training Dataset and evaluating using Validation Dataset

In [ ]:
history = model.fit( train_generator,
                    epochs = 8,
                    steps_per_epoch = len(train_generator),
                    validation_data = validation_generator,
                    validation_steps = len(validation_generator),
                    verbose = 1)

Plotting the Loss and Accuracy against epochs

In [ ]:
loss = history.history['loss']
accuracy = history.history['accuracy']
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(range(1, len(loss) + 1), loss, label='Training Loss')
plt.plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.grid(True)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, len(accuracy) + 1), accuracy, label='Training Accuracy')
plt.plot(range(1, len(val_accuracy) + 1), val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

Saving the trained Model

In [ ]:
model.save('FakeFaceDetectionModel.h5')